## 🔥 Optimización del Rendimiento Energético en una Planta de Vapor

Se desea determinar la asignación óptima de carga entre dos calderas (A y B) que presentan **eficiencia no lineal decreciente** con respecto al flujo de vapor producido, con el objetivo de **minimizar el consumo total de combustible**.

---

### 🔸 Parámetros del problema

| Parámetro        | Caldera A                   | Caldera B                   |
|------------------|-----------------------------|-----------------------------|
| Producción       | $x \in [10,\ 38.7]$ ton/h   | $y \in [20,\ 42.72]$ ton/h  |
| Eficiencia       | $\eta_A(x) = 0.9 - 0.0005x^2$ | $\eta_B(y) = 0.88 - 0.0004y^2$ |
| Calor específico | $h = 4200\ \text{kJ/kg}$    | $h = 4200\ \text{kJ/kg}$    |
| PCI              | $45000\ \text{kJ/kg}$       | $45000\ \text{kJ/kg}$       |
| Demanda total    | $x + y = 60$ ton/h          |                             |

---

### 🔸 Variables

- $x$: flujo de vapor de la caldera A (ton/h)
- $y$: flujo de vapor de la caldera B (ton/h)
- $\eta_A$, $\eta_B$: eficiencias térmicas de las calderas

---

### 🔸 Restricciones

1. Eficiencia de las calderas:

$$
\eta_A = 0.9 - 0.0005x^2 \\
\eta_B = 0.88 - 0.0004y^2
$$

2. Límite mínimo de eficiencia:

$$
\eta_A \geq 0.15 \\
\eta_B \geq 0.15
$$

3. Balance de vapor:

$$
x + y = 60
$$

4. Límites operativos:

$$
10 \leq x \leq 38.7 \\
20 \leq y \leq 42.72
$$

---

### 🔸 Función Objetivo

Minimizar el consumo total de combustible:

$$
\min \left( \frac{x \cdot h}{\eta_A \cdot PCI} + \frac{y \cdot h}{\eta_B \cdot PCI} \right)
$$

---

### 💻 Implementación en Pyomo

In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

# Crear modelo
model = ConcreteModel()

# Parámetros
h = 4200     # kJ/kg
PCI = 45000  # kJ/kg

# Variables de producción
model.x = Var(bounds=(10, 38.7))    # Caldera A
model.y = Var(bounds=(20, 42.72))   # Caldera B

# Eficiencias (expresiones no lineales)
model.eta_A = Expression(expr=0.9 - 0.0005 * model.x ** 2)
model.eta_B = Expression(expr=0.88 - 0.0004 * model.y ** 2)

# Restricción de demanda total
model.bal_vapor = Constraint(expr=model.x + model.y == 60)

# Restricciones de eficiencia mínima
model.eta_A_min = Constraint(expr=model.eta_A >= 0.15)
model.eta_B_min = Constraint(expr=model.eta_B >= 0.15)

# Objetivo: minimizar el consumo total de combustible
model.obj = Objective(
    expr=(model.x * h) / (model.eta_A * PCI) + (model.y * h) / (model.eta_B * PCI),
    sense=minimize
)

# Resolver con IPOPT
solver = SolverFactory('ipopt')
results = solver.solve(model, tee=True)

# Resultados
print("Flujo óptimo caldera A (x):", value(model.x), "ton/h")
print("Flujo óptimo caldera B (y):", value(model.y), "ton/h")
print("Eficiencia A:", value(model.eta_A))
print("Eficiencia B:", value(model.eta_B))
print("Consumo total de combustible:", value(model.obj))

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        2
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
Tot